# Python Analysis Project

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

## ***Reading In and Organizing Data:***

In [353]:
advanced_stats = pd.read_csv('./Baseball Data/Fangraphs Leaderboard (Advanced).csv')
standard_stats = pd.read_csv('./Baseball Data/FanGraphs Leaderboard Batted Ball Stats.csv')

#####################################################################################################################

pitch_info_hm = pd.read_csv('./Baseball Data/FanGraphs Leaderboard (Pitch Info H-Movement).csv')
pitch_info_pd = pd.read_csv('./Baseball Data/FanGraphs Leaderboard (Pitch Info Plate Discipline).csv')
pitch_info_velo = pd.read_csv('./Baseball Data/FanGraphs Leaderboard (Pitch Info Velo).csv')
pitch_info_vm = pd.read_csv('./Baseball Data/FanGraphs Leaderboard(Pitch Info Vertical Movement).csv')
pitch_type = pd.read_csv('./Baseball Data/FanGraphs Leaderboard (Pitch Info Pitch Type).csv')

######################################################################################################################

batted_ball = pd.read_csv('./Baseball Data/FanGraphs Leaderboard Batted Ball Stats.csv')

In [354]:
pd.set_option('display.max_columns', None)

- **Merging Advanced and Standard Stats**

In [355]:
standard_stats = standard_stats.drop(['Team','BABIP','playerid'],axis = 1)
#standard_stats.head()

In [356]:
std_adv = pd.merge(advanced_stats,standard_stats, on = "Name")

In [357]:
std_adv = std_adv.drop('Team',axis = 1)

In [358]:
first_column = std_adv.pop('playerid')
std_adv.insert(0, 'playerid', first_column)

In [359]:
#std_adv.head()

In [360]:
#std_adv.loc[std_adv.Name == 'Aaron Nola']

- **Merging Pitch/fx Data**

In [361]:
#print(list(pitch_type.columns))
#print(list(pitch_info_pd.columns))
#print(list(pitch_info_hm.columns))
#print(list(pitch_info_vm.columns))
#print(list(pitch_info_velo.columns))

In [362]:
pitch = pd.merge(pitch_type,
                 pitch_info_velo.drop(['Name','Team','IP'],axis = 1), on = 'playerid')
pitch = pd.merge(pitch,
                    pitch_info_hm.drop(['Name','Team','IP'],axis = 1),on = 'playerid')
pitch = pd.merge(pitch,
                    pitch_info_vm.drop(['Name','Team','IP'],axis = 1),on = 'playerid')
pitch = pd.merge(pitch,
                    pitch_info_pd.drop(['Name','Team','IP'],axis = 1),on = 'playerid')
first_column = pitch.pop('playerid')
pitch.insert(0, 'playerid', first_column)

In [363]:
first_column = pitch.pop('playerid')
pitch.insert(0, 'playerid', first_column)

In [364]:
pitch = pitch.drop('Team',axis = 1)

In [365]:
pitch = pitch.sort_values(by = 'IP', ascending = False)

In [366]:
#pitch.head()

- **Batted Ball Data**

In [367]:
batted_ball = batted_ball.drop('Team',axis = 1)

In [368]:
first_column = batted_ball.pop('playerid')
batted_ball.insert(0, 'playerid', first_column)

In [369]:
#batted_ball.head()

In [370]:
#std_adv.head()

In [371]:
for feature in list(batted_ball.columns):
    if feature in (list(std_adv.columns)[2:]):
        batted_ball = batted_ball.drop(feature,axis = 1) 

In [372]:
std_adv_bat = pd.merge(std_adv,batted_ball, on = ['playerid','Name'])

In [373]:
#std_adv_bat

- **Incorperating Spin Rate Data**

In [374]:
spin = pd.read_csv('./Baseball Data/spin_rate_2017-2021.csv')

In [375]:
spin['Name'] = [f'{first} {last}' for first,last in zip(spin[list(spin.columns)[1]],spin[list(spin.columns)[0]])]

In [376]:
first_column = spin.pop('Name')
spin.insert(0, 'Name', first_column)

In [377]:
spin = spin.drop(['p_formatted_ip','p_strikeout','p_k_percent','p_bb_percent'
                  ,'player_id','year',' first_name','last_name','Unnamed: 17'], axis = 1)

In [378]:
avg_spin_rates = spin.groupby(by = 'Name').mean()

In [379]:
avg_spin_rates = avg_spin_rates.reset_index()

In [380]:
avg_spin_rates['Name'] = avg_spin_rates['Name'].apply(lambda x: x.strip())

**Savant pitch keys**

- ff = 4 seam
- sl = slider
- ch = changeup
- cu = curveball
- si = sinker
- fc = cut fastball
- fs = split fastball
- kn = knuckleball

**Fangraphs Pitch Keys**

- FA = fourseam or unclassified fastballs
- FT = two seam fastball
- FC = cut fastball
- FS = split fastball
- FO = forkball
- SI = sinker
- SL = slider
- CU = curveball
- KC = knuckle curve
- EP = Ephes
- CH = changeup
- SC = screwball
- KN = knuckleball
- UN = unknowns

In [381]:
pitch_plus = pd.merge(pitch.sort_values(by = 'Name'),avg_spin_rates,on = 'Name')
#pitch_plus.head(10)

In [382]:
missing_player = list(std_adv_bat.loc[~std_adv_bat['Name'].isin(pitch_plus['Name'])]['Name'])[0]

In [383]:
missing_player

'Hyun-Jin Ryu'

In [384]:
std_adv_bat = std_adv_bat.loc[std_adv_bat.Name != missing_player]

**Merging All Data**

In [421]:
master_stats = pd.merge(std_adv_bat,pitch_plus, on = ['playerid','Name'])
master_stats

,playerid,Name,K/9,BB/9,K/BB,HR/9,K%,BB%,K-BB%,AVG,WHIP,BABIP,LOB%,ERA-,FIP-,xFIP-,ERA,FIP,E-F,xFIP,SIERA,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,RS,RS/9,Balls,Strikes,Pitches,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,IP,FA%,FC%,FS%,SI%,CH%,SL%,CU%,CS%,KN%,SB%,XX%,vFA,vFC,vFS,vSI,vCH,vSL,vCU,vCS,vKN,vSB,FA-X,FC-X,FS-X,SI-X,CH-X,SL-X,CU-X,CS-X,KN-X,SB-X,FA-Z,FC-Z,FS-Z,SI-Z,CH-Z,SL-Z,CU-Z,CS-Z,KN-Z,SB-Z,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,Pace,ff_avg_spin,sl_avg_spin,ch_avg_spin,cu_avg_spin,si_avg_spin,fc_avg_spin,fs_avg_spin,kn_avg_spin,fastball_avg_spin
0,10954,Jacob deGrom,11.65,2.05,5.69,0.81,0.331,0.058,0.273,0.200,0.96,0.282,0.804,58,59,64,2.35,2.49,-0.14,2.77,2.95,1.32,0.209,0.450,0.340,0.134,0.113,394,4.53,3853,8185,12038,0.384,0.334,0.282,0.222,0.479,0.299,782.1,0.444,NaN,0.000,0.071,0.141,0.274,0.057,NaN,NaN,NaN,NaN,97.0,NaN,92.2,95.5,89.8,91.6,82.7,NaN,NaN,NaN,-3.4,NaN,1.2,-7.5,-7.8,2.2,3.9,NaN,NaN,NaN,8.6,NaN,3.5,6.0,1.8,2.2,-3.5,NaN,NaN,NaN,0.331,0.689,0.520,0.492,0.777,0.691,0.527,NaN,2401.600000,2513.600000,1689.600000,2603.600000,2277.000000,NaN,NaN,NaN,2393.600000
1,10603,Chris Sale,13.21,1.97,6.69,1.02,0.367,0.055,0.312,0.202,0.97,0.298,0.760,66,59,60,3.08,2.57,0.51,2.63,2.61,1.09,0.204,0.415,0.382,0.114,0.134,283,4.90,2720,5699,8419,0.396,0.332,0.272,0.205,0.488,0.307,519.2,0.374,0.000,0.000,0.117,0.158,0.349,0.000,NaN,NaN,NaN,0.000,94.7,93.3,94.7,92.7,86.5,79.6,81.9,NaN,NaN,NaN,8.7,11.6,10.0,11.3,11.1,-7.4,-8.0,NaN,NaN,NaN,6.0,4.3,6.1,1.7,-0.1,-3.7,-2.8,NaN,NaN,NaN,0.325,0.621,0.486,0.503,0.774,0.691,0.543,NaN,2358.666667,2461.000000,2008.666667,NaN,2238.666667,NaN,NaN,NaN,2332.666667
2,3137,Max Scherzer,12.28,2.20,5.59,1.06,0.344,0.062,0.282,0.198,0.97,0.279,0.808,64,66,74,2.76,2.86,-0.09,3.16,2.93,0.80,0.192,0.359,0.449,0.123,0.115,456,5.36,3951,8335,12286,0.404,0.326,0.270,0.204,0.484,0.312,766.0,0.484,0.080,NaN,0.000,0.148,0.199,0.084,NaN,NaN,NaN,0.000,94.5,89.4,NaN,95.3,84.6,85.7,78.0,NaN,NaN,NaN,-6.2,0.4,NaN,-3.0,-8.5,2.1,6.1,NaN,NaN,NaN,7.4,3.3,NaN,7.9,0.1,0.0,-5.5,NaN,NaN,NaN,0.321,0.674,0.513,0.490,0.768,0.689,0.544,NaN,2476.200000,2366.600000,1482.600000,2638.600000,2486.000000,2477.800000,NaN,NaN,2476.800000
3,15890,Luis Severino,10.60,2.34,4.53,0.91,0.290,0.064,0.226,0.220,1.09,0.292,0.755,73,68,72,3.13,2.99,0.14,3.10,3.28,1.42,0.222,0.457,0.322,0.113,0.122,290,6.58,2141,4312,6453,0.384,0.362,0.254,0.197,0.485,0.317,396.2,0.510,0.001,NaN,NaN,0.136,0.352,0.000,NaN,NaN,NaN,0.000,97.6,95.2,NaN,NaN,87.8,88.1,97.9,NaN,NaN,NaN,-4.3,-2.6,NaN,NaN,-6.8,5.1,-6.2,NaN,NaN,NaN,8.5,6.7,NaN,NaN,5.0,-2.4,9.2,NaN,NaN,NaN,0.281,0.669,0.492,0.528,0.819,0.744,0.545,NaN,2333.000000,2780.500000,1659.500000,NaN,NaN,NaN,NaN,NaN,2333.000000
4,19427,Shane Bieber,11.49,2.11,5.44,1.10,0.314,0.058,0.256,0.231,1.12,0.315,0.779,71,68,68,3.23,3.00,0.23,2.97,3.17,1.36,0.217,0.450,0.332,0.092,0.146,272,4.99,2646,5105,7751,0.390,0.346,0.264,0.143,0.469,0.388,490.1,0.441,0.032,NaN,0.000,0.062,0.227,0.223,NaN,NaN,NaN,NaN,93.3,89.2,NaN,93.9,88.0,84.9,82.7,NaN,NaN,NaN,-5.4,1.0,NaN,-6.7,-8.7,1.0,3.8,NaN,NaN,NaN,9.0,4.4,NaN,7.7,3.2,-0.4,-8.4,NaN,NaN,NaN,0.331,0.618,0.473,0.429,0.837,0.693,0.496,NaN,2305.250000,2463.000000,1691.750000,2281.750000,2189.000000,2598.000000,NaN,NaN,2324.250000
5,2429,Corey Kluber,10.28,1.93,5.33,0.97,0.289,0.054,0.235,0.215,1.01,0.280,0.776,64,70,72,2.85,3.01,-0.16,3.11,3.23,1.30,0.216,0.443,0.341,0.102,0.125,310,5.48,2539,5064,7603,0.394,0.336,0.270,0.192,0.478,0.330,508.2,0.110,0.266,NaN,0.289,0.073,0.245,0.000,NaN,NaN,NaN,0.000,92.4,88.7,NaN,92.2,85.4,83.9,83.6,NaN,NaN,NaN,-3.4,2.1,NaN,-8.4,-7.1,8.9,8.9,NaN,NaN,NaN,7.4,2.3,NaN,3.9,1.4,-0.2,-0.3,NaN,NaN,NaN,0.323,0.629,0.482,0.448,0.846,0.718,0.520,NaN,2391.000000,NaN,1729.000000,2598.000000,2246.500000,2592.000000,NaN,NaN,2402.500000
6,11762,Noah Syndergaard,9.20,2.17,4.25,0.78,0.245,0.058,0.188,0.249,1.21,0.318,0.713,91,73,82,3.67,3.10,0.58,3.51,3.73,1.68,0.216,0.492,0.293,0.101,0.104,227,5.34,1974,3992,5966,0.405,0.349,0.245,0.226,0.512,0.262,38

In [422]:
master_stats = master_stats.dropna(axis=1, how='all', thresh=None, subset=None)

## ***Visualizing Data:***

- **Correlations**

In [423]:
master_stats_corr = pd.DataFrame(master_stats.corr())

In [424]:
fig = px.imshow(master_stats_corr,width=950, height=950,color_continuous_scale = 'spectral',
               title = 'Batted Ball,Advanced, and Standard Stats Heatmap')
fig.show()

## ***Visualizing Pitch Data:***

- **Correlations**

In [428]:
fig = px.scatter(master_stats.sort_values(by = 'FIP').head(25), x="ERA", y="FIP",
                 size="IP", color ="Hard%" ,
                 hover_name="Name", log_x=False, size_max=20, width=900, height=800,
                 title = "")

fig.show()